In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [ ]:
def distance(row): 
    address1 = (row['utente_lat'], row['utente_lon']) 
    address2 = (row['hospital_lat'], row['hospital_lon']) 
    try:
        return (geodesic(address1, address2).kilometers) 
    except: 
        return np.nan

## Other

In [ ]:
utente = pd.read_csv('/opt/airflow/datasets/absenteeism/Utentes.txt', sep='|', encoding="ISO-8859-1", on_bad_lines= "skip")
utente.columns = ['UtenteSkey', 'GeografiaSKey', 'NUM_SEQUENCIAL', 'DataNascimento', 'Sexo', 'Nacionalidade', 'CódigoUnidadeSaude', 'UnidadeSaude']

In [ ]:
geografia = pd.read_parquet('/opt/airflow/datasets/absenteeism/geografia_coord.parquet')

to_fill_na = geografia[geografia.lat!='nan'][['Concelho', 'lat','lon']].drop_duplicates(subset='Concelho', keep='first')

geografia = geografia.merge(to_fill_na, on='Concelho', how='left')
geografia.loc[:,'lat_x'] = np.where(geografia.lat_x=='nan', geografia.lat_y, geografia.lat_x)
geografia.loc[:,'lon_x'] = np.where(geografia.lon_x=='nan', geografia.lat_y, geografia.lon_x)

geografia = geografia.drop(columns=['lat_y','lon_y'])
geografia.columns = ['GeografiaSKey', 'CodDistrito', 'Distirito', 'CodConcelho', 'Concelho',
       'CodFreguesia', 'Freguesia', 'Freguesia Geo', 'Concelho Geo',
       'Distrito Geo', 'lat', 'lon']

del to_fill_na

geografia.columns = ['GeografiaSKey', 'CodDistrito', 'Distirito', 'CodConcelho', 'Concelho',
       'CodFreguesia', 'Freguesia', 'Freguesia Geo', 'Concelho Geo',
       'Distrito Geo', 'utente_lat', 'utente_lon']

geografia[['GeografiaSKey','utente_lat', 'utente_lon']] = geografia[['GeografiaSKey','utente_lat', 'utente_lon']].astype('float')    
geografia[['hospital_lat','hospital_lon']]=[41.1123,  -8.6001]
geografia["dist_btw"] = geografia.apply(lambda row: distance(row), axis = 1 )

In [ ]:
geografia = geografia.merge(geografia[geografia.Concelho!='nan'].groupby('Concelho', as_index=False).agg(dist_btw_c=('dist_btw','median')), on='Concelho', how='inner')
geografia['dist_btw'] = np.where((geografia.dist_btw_c*1.5<geografia.dist_btw) & (geografia.dist_btw>500), geografia.dist_btw_c, geografia.dist_btw)

In [ ]:
geografia = geografia.drop(columns=['dist_btw_c']).merge(geografia[geografia.Concelho!='nan'].groupby('Distirito', as_index=False).agg(dist_btw_c=('dist_btw','median')), on='Distirito', how='inner')
geografia['dist_btw'] = np.where((geografia.dist_btw_c*1.5<geografia.dist_btw) & (geografia.dist_btw>500), geografia.dist_btw_c, geografia.dist_btw)

In [ ]:
geografia[geografia.dist_btw>500].Distirito.unique()

In [ ]:
utente = utente.merge(geografia[['GeografiaSKey', 'Distirito','Concelho', 'utente_lat', 'utente_lon', 'hospital_lat', 'hospital_lon', 'dist_btw']], 
                      on='GeografiaSKey', how='inner')[['NUM_SEQUENCIAL', 'DataNascimento',
       'Sexo', 'Nacionalidade', 'Distirito', 'Concelho',  'utente_lat', 'utente_lon', 'hospital_lat',
       'hospital_lon', 'dist_btw']]
del geografia

#  caracterização utente

In [ ]:
from datetime import datetime

In [ ]:
# validar que Sexo é só feminino e masculino caso contrário isto não se pode fazer assim ...
utente['Sexo']=np.where(utente.Sexo=='Masculino', 1, np.where(utente.Sexo=='Feminino', 0, 2))

utente['Idade'] = np.round((pd.to_datetime(datetime.now())-pd.to_datetime(utente.DataNascimento)).dt.days/365, 0)
utente = utente.drop(columns=['DataNascimento'])

In [ ]:
utente = utente.drop(columns=['Distirito', 'Concelho', 'hospital_lat', 'hospital_lon']) #, 'utente_lat', 'utente_lon'])

In [ ]:
a = utente.groupby('Nacionalidade', as_index=False).agg( volume=('NUM_SEQUENCIAL', 'count')) 
a['volume']=a['volume']/a['volume'].sum()
nacionalidade_to_consider=a[a.volume>=(1/a.shape[0])].Nacionalidade.tolist()

In [ ]:
utente['Nacionalidade']=np.where(utente.Nacionalidade.isin(nacionalidade_to_consider), utente.Nacionalidade, 'Nacionalidade_outras')

In [ ]:
# based on https://www.indexmundi.com/portugal/age_structure.html
conditions = [
    (utente['Idade'] >= 65),
    (utente['Idade'] >= 55) & (utente['Idade'] < 65),
    (utente['Idade'] >= 25) & (utente['Idade'] < 55),
    (utente['Idade'] >= 15) & (utente['Idade'] < 25),
    (utente['Idade'] <  15) | (utente['Idade'].isna())
    ]
faixas = [">=65", "55-64", "25-54", "15-24", "0-14"]
utente["FaixaEtaria"] = np.select(conditions, faixas)

In [ ]:
utente.loc[:, ~utente.columns.isin(['utente_lat', 'utente_lon'])].to_parquet('/opt/airflow/datasets/absenteeism/modelacao_1_train_utente_basic.parquet', compression='gzip')

In [ ]:
utente.loc[:, ~utente.columns.isin(['utente_lat', 'utente_lon'])].head()

In [ ]:
utente.to_parquet('/opt/airflow/datasets/absenteeism/modelacao_1_train_utente_basic_coord.parquet', compression='gzip')

In [ ]:
utente.head()